In [41]:
import numpy as np
import matplotlib.pyplot as plt
def loss(X,y,W):
    A=softmax_stable(X.dot(W))
    id0=range(X.shape[0])
    return -(np.log(A[id0,y]).sum())/X.shape[0]

def softmax_stable(Z):
    e_Z=np.exp(Z-np.max(Z,axis=1,keepdims=True))
    A=e_Z/e_Z.sum(axis=1,keepdims=True)
    return A

def softmax_grad(X,y,W):
    """
    W: 2d numpy array of shape (d,C)
    each column correspoding to one output node
    X: shape (N,d)
    each row is data point
    y: shape(N,)
    
    """
    A=softmax_stable(X.dot(W)) #shape of (N,C)
    id0=range(X.shape[0])
    A[id0,y]-=1
    return X.T.dot(A)/X.shape[0] #shape of (d,C)

def softmax_fit(X,y,W,lr=0.01,nepoches=100,tol=1e-5,batch_size=10):
    W_old=W.copy()
    ep=0
    loss_hist=[loss(X,y,W_old)]
    N=X.shape[0]
    nbatches=int(np.ceil(float(N)/batch_size))
    while ep<=nepoches:
        W_old=W.copy()
        mix_ids=np.random.permutation(N)
        ep+=1
        for i in range(nbatches):
            batch_ids=mix_ids[batch_size*i:min(batch_size*(i+1),N)]
            X_batch=X[batch_ids]
            y_batch=y[batch_ids]
            
            W-=lr*softmax_grad(X_batch,y_batch,W)
            
        loss_hist.append(loss(X,y,W))
        if np.linalg.norm(W-W_old)/W.size<tol:
            break
        
    return (W,loss_hist)

def pred(X,W):
    return np.argmax(X.dot(W),axis=1)


In [42]:
import numpy as np
def load_mnist(prefix,folder):
    intType=np.dtype('int32').newbyteorder('>')
    nMetaDataBytes=4*intType.itemsize
    data=np.fromfile(folder+"/"+prefix+"-images-idx3-ubyte",dtype='ubyte')
    
    magicBytes,nImages,width,height=np.frombuffer(data[:nMetaDataBytes].tobytes(),intType)
    data=data[nMetaDataBytes:].astype(dtype='float32').reshape(nImages,width*height)
    
    labels=np.fromfile(folder+"/"+prefix+"-labels-idx1-ubyte",dtype='ubyte')[2*intType.itemsize:]
    
    return data,labels
X_train,y_train=load_mnist("train","/datasets/mnist/")
X_test,y_test=load_mnist("t10k","/datasets/mnist/")



Xbar_train=np.concatenate((np.ones((X_train.shape[0],1)),X_train),axis=1)
Xbar_test=np.concatenate((np.ones((X_test.shape[0],1)),X_test),axis=1)

In [43]:
W_init=np.random.rand(Xbar_train.shape[1],10)
(W,loss_hist)=softmax_fit(Xbar_train,y_train,W_init)
y_pred=pred(Xbar_test,W)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


In [48]:
accuracy=(np.equal(y_pred,y_test).sum()/y_test.shape[0])
print("Accuracy of model {}%".format(accuracy*100))

Accuracy of model 92.43%


In [39]:
A=softmax_stable(Xbar_train.dot(W))
print()

[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]


# Dùng thư viện sklearn


In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model=LogisticRegression(C=1e5,solver='lbfgs',multi_class='multinomial')
model=model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print("Accuracy of model {}%".format(accuracy_score(y_test,y_pred.tolist())*100))


Accuracy of model 92.43%


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
